In [2]:
#Import libraries
import pandas as pd
import numpy as np

In [3]:
#Open csv source/Users/baodinhgiatran/Desktop/DATA 490/
data = pd.read_csv("Raw Data/usage_dataset_20230306.csv")

In [4]:
remove_values = [29200, 25400, 17500]
for i in remove_values:
    data = data.loc[data['usage'] != i]

In [5]:
#Aggregate usage by unique id
df_usage = data.groupby(["unique_identifier"]).agg({ 'usage':'sum'})

In [6]:
#Filter active firms
df_active = data[data['status'] == 'ACTIVE' ][["unique_identifier","status"]].drop_duplicates()
df_active

,unique_identifier,status
0,2ac7d64df7018a4137d7c5cf98c40061,ACTIVE
6,7de37d6386e8af4fda4a965766136ad1,ACTIVE
7,4dc18e24bce7f6c1894400fb7b25f450,ACTIVE
8,019b70147b78a066e76e6e9cb7cc40bb,ACTIVE
11,4f478a7903a1acdba35ab3cb60791cc5,ACTIVE
...,...,...
66477,54ed7b0e1f4572666cb548b2c128f86c,ACTIVE
66478,fe62ab58ca32c0debd6191cac35f31e2,ACTIVE
66484,fc18b8f49d252f5d5ccfcfca7a00301c,ACTIVE
66486,4c6585f044d758138beb357489213ebb,ACTIVE


In [7]:
#Filter and Remove active firms from finalled firms dataframe
active_list = df_active['unique_identifier'].values.tolist()
df_finalled= data[data['status'] == 'FINALLED' ][["unique_identifier","status"]].drop_duplicates()
df_finalled = df_finalled[~df_finalled['unique_identifier'].isin(active_list)]
df_finalled

,unique_identifier,status
1,26fe8b75ad11c751336e76ef00aa9c29,FINALLED
3,afcd5e3beba66fb8ca35625557de98af,FINALLED
4,e7df36f919ffd1d6ab666ac6edd995aa,FINALLED
5,309e4667ec44b5ff789b05075637fd2f,FINALLED
9,85dc06d71276a5802eec502e21095be9,FINALLED
...,...,...
66482,76ba4ae7ee025351862d5eb5f2911c14,FINALLED
66483,f1b2e0d475c77ad5a02659aa10d43dda,FINALLED
66485,82ed5158205f7a8a94211c7dd3ed44bb,FINALLED
66488,707c661192c31ea9a2a69c92e647a81b,FINALLED


In [8]:
#Filter and Remove active and finalled firms from expect finalled firms dataframe
finalled_list = df_finalled['unique_identifier'].values.tolist()
value_list = active_list + finalled_list
df_expect = data[data['status'] == 'EXPECT FINALLED' ][["unique_identifier","status"]].drop_duplicates()
df_expect = df_expect[~df_expect['unique_identifier'].isin(value_list)]

In [9]:
#Append status list
df_status = pd.concat([df_active,df_finalled,df_expect], ignore_index=True)
df_status

,unique_identifier,status
0,2ac7d64df7018a4137d7c5cf98c40061,ACTIVE
1,7de37d6386e8af4fda4a965766136ad1,ACTIVE
2,4dc18e24bce7f6c1894400fb7b25f450,ACTIVE
3,019b70147b78a066e76e6e9cb7cc40bb,ACTIVE
4,4f478a7903a1acdba35ab3cb60791cc5,ACTIVE
...,...,...
49985,fe41f1743823cba01e40420b2fa6833c,EXPECT FINALLED
49986,bd06b0cd9132defd423aefa52408dfd5,EXPECT FINALLED
49987,ae782d21ce800df446e3862e9b387d61,EXPECT FINALLED
49988,215b411b68a49380ce6c5375747870c9,EXPECT FINALLED


In [12]:
#Create a clean data frame of usage csv
df_clean = df_usage.merge(df_status, on='unique_identifier', how='left')
df_filter = df_clean.loc[(df_clean['usage'] >= 100) & (df_clean['usage'] <= 5000000)]

In [13]:
df_filter

,unique_identifier,usage,status
0,0001230a214b39e0e5c463bfe440fb15,81440.0,FINALLED
1,000345e997e72b61b990d2689c76427f,556.3,ACTIVE
2,0003c4d7aeb24f319f0d7c6ddb60bb8f,32564.0,FINALLED
3,00082675e86a9f3cf5fdcc5d4cd9114d,5519.0,FINALLED
4,00095201031df44962513f378842d521,5946.0,ACTIVE
...,...,...,...
49985,fff7c0d1b2f896b1018ef67a9d286361,11980.0,FINALLED
49986,fffce8918ebaae88423f62806f22c414,11080.0,ACTIVE
49987,fffd155082881fa090b08ca6ceed7005,93876.0,ACTIVE
49988,fffe3fcb6ca0166ec15af3958ac145e8,9072.0,ACTIVE


In [14]:
#Export to csv
df_filter.to_csv('Clean Data/usage_clean_20230420.csv', index = False)